# Mini-projet : Segmentation client (Marketing Campaign)

Ce notebook implémente pas à pas les consignes du PDF :
1) Nettoyage des données
2) Création de variables
3) Analyse descriptive simple
4) Préprocessing (encodage + standardisation)
5) Réduction de dimension (PCA)
6) Clustering (KMeans) et interprétation
   

## 0. Imports et chargement des données

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from mpl_toolkits.mplot3d import Axes3D  # nécessaire pour la projection 3D
from datetime import datetime

# Chemin du CSV (déposé par l'utilisateur)
csv_path = '/Users/suewernberg/Desktop/DATABOOTCAMP/DATABOOTCAMP_SUZANNE.WERNBERG-MOLLER/Rush4/Camp_Market.csv'
df = pd.read_csv(csv_path)

print('Aperçu des 5 premières lignes:')
display(df.head())
print('\nInfos:')
print(df.info())

Aperçu des 5 premières lignes:


,ID;Year_Birth;Education;Marital_Status;Income;Kidhome;Teenhome;Dt_Customer;Recency;MntWines;MntFruits;MntMeatProducts;MntFishProducts;MntSweetProducts;MntGoldProds;NumDealsPurchases;NumWebPurchases;NumCatalogPurchases;NumStorePurchases;NumWebVisitsMonth;AcceptedCmp3;AcceptedCmp4;AcceptedCmp5;AcceptedCmp1;AcceptedCmp2;Complain;Z_CostContact;Z_Revenue;Response
0,5524;1957;Graduation;Single;58138;0;0;2012-09-...
1,2174;1954;Graduation;Single;46344;1;1;2014-03-...
2,4141;1965;Graduation;Together;71613;0;0;2013-0...
3,6182;1984;Graduation;Together;26646;1;0;2014-0...
4,5324;1981;PhD;Married;58293;1;0;2014-01-19;94;...



Infos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 1 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                    Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                                                                                                                                                                                    --------------  ----- 
 0   ID;Year_Birth;Education;Marital_Status;Income;Kidhome;Teenhome;Dt_Customer;Recency;MntWines;MntFruits

## 1. Nettoyage de la base de données

In [4]:
# Conversion de la date d'enrôlement client
if 'Dt_Customer' in df.columns:
    df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], errors='coerce')

# Gestion simple des valeurs manquantes : suppression des lignes incomplètes
df = df.dropna()

# Suppression de valeurs aberrantes simples sur le revenu (1er-99e percentile)
if 'Income' in df.columns:
    q_low = df['Income'].quantile(0.01)
    q_hi  = df['Income'].quantile(0.99)
    df = df[(df['Income'] > q_low) & (df['Income'] < q_hi)]

# Renommer pour plus de lisibilité
df = df.rename(columns={
    'MntWines':'Wines',
    'MntFruits':'Fruits',
    'MntMeatProducts':'Meat',
    'MntFishProducts':'Fish',
    'MntSweetProducts':'Sweet',
    'MntGoldProds':'Gold'
})

print('Après nettoyage:')
display(df.head())
print(df.shape)

Après nettoyage:


,ID;Year_Birth;Education;Marital_Status;Income;Kidhome;Teenhome;Dt_Customer;Recency;MntWines;MntFruits;MntMeatProducts;MntFishProducts;MntSweetProducts;MntGoldProds;NumDealsPurchases;NumWebPurchases;NumCatalogPurchases;NumStorePurchases;NumWebVisitsMonth;AcceptedCmp3;AcceptedCmp4;AcceptedCmp5;AcceptedCmp1;AcceptedCmp2;Complain;Z_CostContact;Z_Revenue;Response
0,5524;1957;Graduation;Single;58138;0;0;2012-09-...
1,2174;1954;Graduation;Single;46344;1;1;2014-03-...
2,4141;1965;Graduation;Together;71613;0;0;2013-0...
3,6182;1984;Graduation;Together;26646;1;0;2014-0...
4,5324;1981;PhD;Married;58293;1;0;2014-01-19;94;...


(2240, 1)


## 2. Création des nouvelles variables

In [5]:
current_year = datetime.now().year
df['Age'] = current_year - df['Year_Birth']

# Total dépensé sur 2 ans
prods = ['Wines','Fruits','Meat','Fish','Sweet','Gold']
df['Spent'] = df[prods].sum(axis=1)

# Statut marital simplifié
def simplify_marital(x):
    if x in ['Married', 'Together']:
        return 'En couple'
    else:
        return 'Célibataire'
df['Living_with'] = df['Marital_Status'].apply(simplify_marital)

# Enfants, taille du foyer, statut parent
df['Children'] = df['Kidhome'] + df['Teenhome']
df['Family_size'] = df['Children'] + np.where(df['Living_with'] == 'En couple', 2, 1)
df['Is_parent'] = np.where(df['Children'] > 0, 1, 0)

# Education simplifiée
def simplify_edu(x):
    s = str(x)
    if ('PhD' in s) or ('Master' in s):
        return 'PostGrad'
    elif 'Graduation' in s:
        return 'Grad'
    else:
        return 'UnderGrad'
df['Education2'] = df['Education'].apply(simplify_edu)

# Ancienneté client (en jours)
max_date = df['Dt_Customer'].max()
df['Customer_for'] = (max_date - df['Dt_Customer']).dt.days

display(df[['Age','Spent','Living_with','Children','Family_size','Is_parent','Education2','Customer_for']].head())

KeyError: 'Year_Birth'

## 3. Analyse descriptive simple

In [ ]:
# Histogramme de l'âge
plt.figure(figsize=(8,4))
plt.hist(df['Age'], bins=30)
plt.title("Distribution de l'âge")
plt.xlabel('Âge')
plt.ylabel('Fréquence')
plt.show()

# Boxplot du revenu
plt.figure(figsize=(8,2.8))
plt.boxplot(df['Income'].dropna(), vert=False)
plt.title('Boîte à moustaches du revenu')
plt.xlabel('Revenu annuel')
plt.show()

# Histogramme des dépenses totales
plt.figure(figsize=(8,4))
plt.hist(df['Spent'], bins=30)
plt.title('Dépenses totales (2 ans)')
plt.xlabel('Montant')
plt.ylabel('Fréquence')
plt.show()

## 4. Préprocessing (encodage + standardisation)

In [ ]:
# Encodage simple des variables catégorielles nécessaires
le_living = LabelEncoder()
le_edu = LabelEncoder()
df['Living_with_enc'] = le_living.fit_transform(df['Living_with'])
df['Education2_enc'] = le_edu.fit_transform(df['Education2'])

# Variables pour le clustering (sans variables de promo)
features = ['Age','Income','Spent','Living_with_enc','Children','Family_size','Is_parent','Education2_enc','Customer_for']
X = df[features].copy()

# Standardisation
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print('Taille de X :', X.shape)

## 5. Réduction de dimension (PCA à 3 composantes)

In [ ]:
pca = PCA(n_components=3, random_state=0)
X_pca = pca.fit_transform(X_scaled)
explained = pca.explained_variance_ratio_
print('Variance expliquée par composante :', explained)
print('Variance expliquée cumulée :', explained.cumsum())

## 6. Clustering (KMeans) et visualisation 3D

In [ ]:
# KMeans avec k=4 (simple pour débuter)
kmeans = KMeans(n_clusters=4, random_state=0, n_init='auto')
clusters = kmeans.fit_predict(X_pca)
df['Cluster'] = clusters

# Nuage de points 3D de la PCA, coloré par cluster
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_pca[:,0], X_pca[:,1], X_pca[:,2], c=clusters)
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.set_title('Projection PCA (3D) avec KMeans (k=4)')
plt.show()

# Répartition des clients par cluster
display(df['Cluster'].value_counts().rename('effectif'))
display((df['Cluster'].value_counts(normalize=True)*100).round(2).rename('%'))

## 7. Interprétation simple des clusters

In [ ]:
summary_cols = ['Age','Income','Spent','Children','Customer_for','Family_size','Is_parent']
grouped_mean = df.groupby('Cluster')[summary_cols].mean().round(1)
display(grouped_mean)

print('\nCommentaires rapides :')
print('- Compare les moyennes par cluster pour repérer les profils (ex: plus haut revenu, plus forte dépense, etc.).')
print('- Observe si un cluster regroupe des familles (Children/Family_size élevés), ou des célibataires (Is_parent=0, Family_size faible).')
print('- Tu peux aussi croiser avec les variables de campagne (AcceptedCmp1..5, Response) si tu veux étudier la réactivité par cluster.')